In [50]:
import re
from tqdm import tqdm
import os,glob
import praw
from praw.models import Submission
from praw.exceptions import ClientException
from prawcore.exceptions import Forbidden,NotFound,ServerError

reddit = praw.Reddit(client_id='ACEORGSlEeJyQhPyNRAUiA',
                     client_secret='xGN33fztXu_4jiOek_RHUtMGhHXAcQ',
                     user_agent='me',
                    password='redditC0bintr@sena',
                    username='Western-Wishbone573')
print(reddit.user.me())

Western-Wishbone573


In [45]:
from convokit import Corpus#, Speaker, download
AVAILABLE_CORP = ['full_cmv','matched_cmv_subsample','gen_reddit']
corp_name_to_filepath = {
                         'full_cmv': '/u/scr/yiweil/my-cmv-corpus_1-1-2010_to_09-24-2021',
                         'matched_cmv_subsample': '/u/scr/yiweil/subsampled_cmv_corpus',
                         'gen_reddit':'/u/scr/yiweil/datasets/convokit/reddit-corpus-small',
                         'switchboard':'/sailhome/yiweil/.convokit/downloads/switchboard-corpus'
                        }

In [46]:
corp_to_analyze = {'matched_cmv_subsample': None}#,'gen_reddit': None,'switchboard':None}
#corp_to_analyze.update({'matched_cmv_subsample': None})

for corp_name in corp_to_analyze:
    if corp_to_analyze[corp_name] is None:
        print("Loading corpus: {}...".format(corp_name))
        corp_to_analyze[corp_name] = Corpus(filename=corp_name_to_filepath[corp_name])
        corp_to_analyze[corp_name].print_summary_stats()
        print('\n')
        
        # Update full meta for conversations if corpus is not `matched_cc_cmv_subsample` (contains comments only, by design)
        if corp_name in {'full_cmv','matched_cmv_subsample','matched_cc_cmv_subsample','gen_reddit','switchboard'}:
            for utt in corp_to_analyze[corp_name].iter_utterances():
                if utt.id == utt.conversation_id: # not a comment but a post
                    convo = corp_to_analyze[corp_name].get_conversation(utt.id)
                    meta_dict = corp_to_analyze[corp_name].get_utterance(utt.id).meta
                    for key in meta_dict:
                        corp_to_analyze[corp_name].get_conversation(utt.id).add_meta(key, meta_dict[key])
                    convo.speaker = utt.speaker
                        
        print("Filtering {}...".format(corp_name))
        corp_to_analyze[corp_name] = filter_bots_mods_removed(corp_to_analyze[corp_name])
        print('\n')
        
        if corp_name in {'matched_cmv_subsample','matched_cc_cmv_subsample'}:
            print("Filtering out generic utterances from {}...".format(corp_name))
            corp_to_analyze[corp_name] = filter_generic_comments(
                corp_to_analyze[corp_name], 
                GENERIC_REGEX_PATS,
                fname=f'generic_dict_{corp_name}.pkl')
            corp_to_analyze[corp_name].print_summary_stats()
            print('\n')
            
        print("Removing footnotes and in-line embedded text from {}...".format(corp_name))
        for utt in tqdm(corp_to_analyze[corp_name].iter_utterances()):
            if utt.id != utt.conversation_id:
                utt.text = cleanup(utt.text)
                
        print("Annotating with climate change information...")
        for convo in tqdm(corp_to_analyze[corp_name].iter_conversations()):
            if convo.id not in c_id2other_cc_label:
                c_id2other_cc_label[convo.id] = 'non-cc'
        for utt in tqdm(corp_to_analyze[corp_name].iter_utterances()):
            utt.meta['is_cc_related_gold'] = utt.conversation_id in climate_convo_idxs_gold
            utt.meta['other_cc_label'] = c_id2other_cc_label[utt.conversation_id]
        for convo in tqdm(corp_to_analyze[corp_name].iter_conversations()):
            convo.meta['other_cc_label'] = c_id2other_cc_label[convo.id]
        print("Distribution of climate change macro labels:\n")
        print("\t",Counter([utt.meta['is_cc_related_gold'] 
                            for utt in tqdm(corp_to_analyze[corp_name].iter_utterances())]))
        print("Distribution of climate change micro labels:")
        print("\t",Counter([utt.meta['other_cc_label'] 
                            for utt in tqdm(corp_to_analyze[corp_name].iter_utterances())]))

Loading corpus: matched_cmv_subsample...
Number of Speakers: 43271
Number of Utterances: 205084
Number of Conversations: 44346


Filtering matched_cmv_subsample...


NameError: name 'filter_bots_mods_removed' is not defined

# Get all posts from r/DeltaLog via PushShift

In [47]:
import pandas as pd
from pmaw import PushshiftAPI
api = PushshiftAPI()

In [48]:
import datetime as dt
before = int(dt.datetime(2022,1,10,0,0).timestamp())
after = int(dt.datetime(2013,1,1,0,0).timestamp())

In [121]:
subreddit="nba"
limit=100000
gen = api.search_submissions(subreddit=subreddit,
                             before=before,
                             after=after)
submissions = list(gen)
print(f'Retrieved {len(submissions)} posts from Pushshift')

INFO:pmaw.PushshiftAPIBase:0 result(s) available in Pushshift
Retrieved 0 posts from Pushshift


In [116]:
pickle.dump(submissions,open('/nlp/u/yiweil/pushshift_deltalog_posts.pkl','wb'))

In [109]:
# limit=10000000
# cmv_gen = api.search_submissions(subreddit='ChangeMyView',
#                              before=before,
#                              after=after)
# cmv_submissions = list(cmv_gen)
# print(f'Retrieved {len(cmv_submissions)} posts from Pushshift')

import pickle
pickle.dump(cmv_submissions,open('/nlp/u/yiweil/pushshift_cmv_posts.pkl','wb'))

In [117]:
len(submissions)

29710

In [110]:
cmv_submissions[0]

{'author': 'Thiek',
 'author_created_utc': 1377312695,
 'author_flair_css_class': None,
 'author_flair_text': None,
 'author_fullname': 't2_cvgjk',
 'created_utc': 1413981942,
 'domain': 'self.changemyview',
 'full_link': 'https://www.reddit.com/r/changemyview/comments/2jzkdn/cmv_those_houses_that_give_out_entire_candy_bars/',
 'gilded': 0,
 'id': '2jzkdn',
 'is_self': True,
 'media_embed': {},
 'num_comments': 0,
 'over_18': False,
 'permalink': '/r/changemyview/comments/2jzkdn/cmv_those_houses_that_give_out_entire_candy_bars/',
 'retrieved_on': 1441110238,
 'score': 1,
 'secure_media_embed': {},
 'selftext': '',
 'stickied': False,
 'subreddit': 'changemyview',
 'subreddit_id': 't5_2w2s8',
 'thumbnail': 'default',
 'title': 'CMV: "Those houses" that give out entire candy bars at Halloween are a total waste. Trick or treaters would be better served targeting houses that give out handfuls of small candy bars than waiting in line for a large one.',
 'url': 'http://www.reddit.com/r/chang

In [111]:
len(corp_to_analyze['matched_cmv_subsample'].get_conversation_ids())

44346

# Scrape comments awarding delta using regex on post text

In [90]:
def get_comments_awarding_deltas(deltalog_post, 
                                pat=re.compile(r"/r/changemyview/comments/(.*?)/(.*?)/(.*?)/")):
    matches = re.findall(pat, deltalog_post['selftext'])
    if len(matches) > 0:
        cmv_post_id = matches[0][0]
        delta_awarding_comment_ids = [m[-1] for m in matches]
        return (cmv_post_id, delta_awarding_comment_ids)
    else:
        return None, None

In [119]:
if not os.path.exists('scraped_deltalog'):
    os.mkdir('scraped_deltalog')

for post in tqdm(submissions):
    cmv_post_id, comment_ids = get_comments_awarding_deltas(post)
    if comment_ids != None:
        if len(comment_ids) == 0:
            print(cmv_post_id)
        #print(cmv_post_id, len(comment_ids))
        with open(os.path.join('scraped_deltalog',f"{post['id']}.txt"),'w') as f:
            f.write(cmv_post_id+'\n')
            f.write('\t'.join(comment_ids))

100%|██████████| 29710/29710 [00:32<00:00, 902.38it/s] 


In [72]:
deltalog_posts = glob.glob('scraped_deltalog/*.txt')
deltalog_posts[:3]

['scraped_deltalog/rrn70z.txt',
 'scraped_deltalog/64wr6v.txt',
 'scraped_deltalog/6124zq.txt']

In [100]:
cmv_post_ids = []
for fname in tqdm(deltalog_posts):
    with open(fname,'r') as f:
        try:
            cmv_post_ids.append(f.read().splitlines()[0])
        except IndexError:
            print(f'{fname} is empty')

  1%|          | 192/29710 [00:00<00:16, 1814.00it/s]

scraped_deltalog/6124zq.txt is empty
scraped_deltalog/5u2hxl.txt is empty
scraped_deltalog/5was9d.txt is empty
scraped_deltalog/5n4m8n.txt is empty
scraped_deltalog/5urj7b.txt is empty
scraped_deltalog/61cst3.txt is empty
scraped_deltalog/5v2qwb.txt is empty
scraped_deltalog/5wkwia.txt is empty
scraped_deltalog/5raqnx.txt is empty
scraped_deltalog/6nu49y.txt is empty
scraped_deltalog/5xv04e.txt is empty
scraped_deltalog/5yfpr7.txt is empty
scraped_deltalog/61u4wr.txt is empty
scraped_deltalog/6207wz.txt is empty
scraped_deltalog/5vk8lv.txt is empty
scraped_deltalog/5mzxbi.txt is empty


  3%|▎         | 823/29710 [00:00<00:13, 2074.98it/s]

scraped_deltalog/60s04j.txt is empty
scraped_deltalog/5owcxn.txt is empty
scraped_deltalog/5vbook.txt is empty
scraped_deltalog/5s64jd.txt is empty
scraped_deltalog/5nofx6.txt is empty
scraped_deltalog/5y8f32.txt is empty
scraped_deltalog/5nsjsi.txt is empty
scraped_deltalog/5qw3tb.txt is empty
scraped_deltalog/5t4k6x.txt is empty
scraped_deltalog/5u4elk.txt is empty
scraped_deltalog/5xogoq.txt is empty
scraped_deltalog/60bar8.txt is empty
scraped_deltalog/5tecwe.txt is empty
scraped_deltalog/5mzvx8.txt is empty
scraped_deltalog/5ytlxa.txt is empty
scraped_deltalog/60qy3n.txt is empty
scraped_deltalog/60b7tp.txt is empty
scraped_deltalog/5wq3n9.txt is empty
scraped_deltalog/5pyovp.txt is empty
scraped_deltalog/5w8zfk.txt is empty
scraped_deltalog/5vh3kf.txt is empty
scraped_deltalog/5psv1z.txt is empty
scraped_deltalog/5zobmo.txt is empty


  3%|▎         | 1031/29710 [00:00<00:14, 1990.46it/s]

...

scraped_deltalog/5w8fpo.txt is empty
scraped_deltalog/615jyr.txt is empty
scraped_deltalog/5u01jr.txt is empty
scraped_deltalog/5rebcj.txt is empty
scraped_deltalog/5oxxoz.txt is empty
scraped_deltalog/5pkivr.txt is empty
scraped_deltalog/5swx8a.txt is empty
scraped_deltalog/5s7xib.txt is empty
scraped_deltalog/5wy1q4.txt is empty
scraped_deltalog/60rtzl.txt is empty
scraped_deltalog/5nwr5p.txt is empty
scraped_deltalog/5u5etj.txt is empty
scraped_deltalog/5sloyf.txt is empty
scraped_deltalog/5vtgec.txt is empty
scraped_deltalog/5vs4s8.txt is empty
scraped_deltalog/5uoqij.txt is empty
scraped_deltalog/5t4bp0.txt is empty
scraped_deltalog/61vs9n.txt is empty
scraped_deltalog/5xigng.txt is empty
scraped_deltalog/5pcti6.txt is empty


 99%|█████████▉| 29443/29710 [00:16<00:00, 1842.75it/s]

scraped_deltalog/5qims0.txt is empty
scraped_deltalog/60ln80.txt is empty
scraped_deltalog/60fj7g.txt is empty
scraped_deltalog/5ugy1o.txt is empty
scraped_deltalog/5u6fao.txt is empty
scraped_deltalog/5yso5v.txt is empty
scraped_deltalog/5wcwbl.txt is empty
scraped_deltalog/61v1i2.txt is empty
scraped_deltalog/60sp59.txt is empty
scraped_deltalog/5qr950.txt is empty
scraped_deltalog/61673k.txt is empty
scraped_deltalog/5stvtn.txt is empty
scraped_deltalog/5q2zn2.txt is empty
scraped_deltalog/5pb7ak.txt is empty
scraped_deltalog/61oyi7.txt is empty
scraped_deltalog/7asnql.txt is empty
scraped_deltalog/5zo7yn.txt is empty
scraped_deltalog/5psjq6.txt is empty
scraped_deltalog/61gvif.txt is empty
scraped_deltalog/5spc0b.txt is empty


100%|██████████| 29710/29710 [00:16<00:00, 1761.56it/s]

scraped_deltalog/5sos09.txt is empty
scraped_deltalog/60qf0y.txt is empty
scraped_deltalog/5xj9gl.txt is empty
scraped_deltalog/5q4g40.txt is empty
scraped_deltalog/5xwahz.txt is empty
scraped_deltalog/617qs0.txt is empty
scraped_deltalog/5wj5g2.txt is empty
scraped_deltalog/5vm5a2.txt is empty
scraped_deltalog/5x3tg0.txt is empty
scraped_deltalog/5uzka9.txt is empty
scraped_deltalog/5v985t.txt is empty
scraped_deltalog/5p35ml.txt is empty
scraped_deltalog/7gitdr.txt is empty
scraped_deltalog/5oxa2w.txt is empty
scraped_deltalog/5ywd68.txt is empty


In [112]:
len(cmv_post_ids)

28456

In [113]:
cmv_post_ids[:3]

['rrmvoo', '64tchq', 'pry2li']

In [114]:
len(set(cmv_post_ids).intersection(set([sub['id'] for sub in cmv_submissions])))

22325

In [103]:
reformatted_cmv_post_ids = ['t3_' + x for x in cmv_post_ids]
reformatted_cmv_post_ids[:3]

['t3_rrmvoo', 't3_64tchq', 't3_pry2li']

In [76]:
corp_to_analyze['matched_cmv_subsample'].get_conversation_ids()[:3]

['t3_psu2sg', 't3_bvupnz', 't3_2oyi3y']

In [104]:
len(set(corp_to_analyze['matched_cmv_subsample'].get_conversation_ids()).intersection(
set(reformatted_cmv_post_ids)))

9184

In [105]:
len(set(corp_to_analyze['matched_cmv_subsample'].get_conversation_ids()))

44346

In [4]:
post = reddit.submission(id=p_id)

In [5]:
post.title

'Deltas awarded in "CMV: I am turning anti-vax or at least anti-mandate because of my personal experi...'

In [8]:
post.selftext

'Below is a list of the deltas awarded in [this post](/r/changemyview/comments/s0p5mu/cmv_i_am_turning_antivax_or_at_least_antimandate/).\n\nPlease note that a change of view [is not necessarily a reversal](https://www.reddit.com/r/changemyview/wiki/index#wiki_what_is_a_.27view.27.3F), and that OP awarding a delta doesn\'t mean the conversation has ended.\n\nFor a full explanation of the delta system, [see here](https://www.reddit.com/r/changemyview/wiki/deltasystem).\n\n-----\n\n# Deltas from OP /u/BatHouseBathHouse\n\n* 1 delta from OP to /u/thelaughterdied for ["The people on this thread are saying it is a coincidence but that doesn\'t mean the CDC is. I have to..."](/r/changemyview/comments/s0p5mu/cmv_i_am_turning_antivax_or_at_least_antimandate/hs4npuv/?context=3)\n\n* 1 delta from OP to /u/edwardlleandre for ["&gt;My uncle just got his booster shot and started having a seizure just hours later despite never h..."](/r/changemyview/comments/s0p5mu/cmv_i_am_turning_antivax_or_at_lea

In [19]:
[(m.start(0), m.end(0), 
  post.selftext[m.start(0):m.end(0)+7]) for m in re.finditer(r'/r/changemyview/comments/*/*/', post.selftext)]

[(53, 78, '/r/changemyview/comments/s0p5mu/'),
 (666, 691, '/r/changemyview/comments/s0p5mu/'),
 (919, 944, '/r/changemyview/comments/s0p5mu/'),
 (1165, 1190, '/r/changemyview/comments/s0p5mu/')]

In [ ]:
r'Part 1\.(.*?)Part 3'

In [44]:
get_comments_awarding_deltas(post)

['hs4npuv', 'hs4nu2e', 'hs4o62b']

In [41]:
comment = reddit.comment(id='hs4npuv')

In [42]:
comment.__dict__

{'_replies': [],
 '_submission': None,
 'id': 'hs4npuv',
 '_reddit': <praw.reddit.Reddit at 0x7fb1b033ee48>,
 '_fetched': True,
 'subreddit_id': 't5_2w2s8',
 'approved_at_utc': None,
 'author_is_blocked': False,
 'comment_type': None,
 'edited': False,
 'mod_reason_by': None,
 'banned_by': None,
 'ups': 1,
 'num_reports': None,
 'author_flair_type': 'text',
 'total_awards_received': 0,
 'subreddit': Subreddit(display_name='changemyview'),
 'author_flair_template_id': None,
 'likes': None,
 'user_reports': [],
 'saved': False,
 'banned_at_utc': None,
 'mod_reason_title': None,
 'gilded': 0,
 'archived': False,
 'collapsed_reason_code': None,
 'no_follow': True,
 'author': Redditor(name='BatHouseBathHouse'),
 'can_mod_post': False,
 'send_replies': True,
 'parent_id': 't1_hs44n3n',
 'score': 1,
 'author_fullname': 't2_7506r',
 'report_reasons': None,
 'removal_reason': None,
 'approved_by': None,
 'all_awardings': [],
 'body': "Δ\n\nThat's correct and I think it's the biggest flaw in wha

In [7]:
post.__dict__

{'comment_limit': 2048,
 'comment_sort': 'confidence',
 'id': 's0x9gy',
 '_reddit': <praw.reddit.Reddit at 0x7fb1d62f7518>,
 '_fetched': True,
 '_comments_by_id': {},
 'approved_at_utc': None,
 'subreddit': Subreddit(display_name='DeltaLog'),
 'selftext': 'Below is a list of the deltas awarded in [this post](/r/changemyview/comments/s0p5mu/cmv_i_am_turning_antivax_or_at_least_antimandate/).\n\nPlease note that a change of view [is not necessarily a reversal](https://www.reddit.com/r/changemyview/wiki/index#wiki_what_is_a_.27view.27.3F), and that OP awarding a delta doesn\'t mean the conversation has ended.\n\nFor a full explanation of the delta system, [see here](https://www.reddit.com/r/changemyview/wiki/deltasystem).\n\n-----\n\n# Deltas from OP /u/BatHouseBathHouse\n\n* 1 delta from OP to /u/thelaughterdied for ["The people on this thread are saying it is a coincidence but that doesn\'t mean the CDC is. I have to..."](/r/changemyview/comments/s0p5mu/cmv_i_am_turning_antivax_or_at_le